In [ ]:
# import 필수
import pandas as pd
import sklearn.model_selection # stratifiedkfold, leaveoneout
import numpy as np
import sys
# from scipy.stats import zscore

import sklearn.svm # SVC / NuSVC / LinearSVC
import sklearn.linear_model # RidgeClassifierCV / LogisticRegressionCV / SGDClassifier

import sklearn.discriminant_analysis
from lightgbm import LGBMClassifier 

from filter_method_fisher import feature_selector

from collections import Counter


Randomstate_val = 42

In [ ]:
Filenames_load = [
    'MS_1-40_ALLepoch_ms4_3group_(1).xlsx',
    'MS_1-40_ALLepoch_ms4_3group_(2).xlsx',
    'MS_1-40_ALLepoch_ms4_3group_(3).xlsx'
                    ]
Filenames_save = [
    'MS_1-40_ALLepoch_ms4_3group_loo_(1).xlsx',
    'MS_1-40_ALLepoch_ms4_3group_loo_(2).xlsx',
    'MS_1-40_ALLepoch_ms4_3group_loo_(3).xlsx'
            ]

for filename,savename in zip(Filenames_load, Filenames_save):
    Dataset_filename = filename
    Dataset= pd.read_excel(Dataset_filename)

    numaric_col = Dataset.columns[2:]

    X, y = Dataset.iloc[:,2:] , Dataset.iloc[:,1]

    cw = "balanced" # balanced/None

    Model_svc_linear = sklearn.svm.SVC(kernel='linear', class_weight = cw, random_state=Randomstate_val)

    Model_ensemble_randomforest_gini = sklearn.ensemble.RandomForestClassifier(class_weight = cw,random_state=Randomstate_val, criterion='gini')

    Model_discriminant_QDA = sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis()

    cv= sklearn.model_selection.LeaveOneOut()

    cvv = cv
    fn = 28
    method = 'fisher'

    svc_linear = DHL_feature_selector()
    svc_linear.filter_method( est=Model_svc_linear, X=X, y=y, cv=cvv, meth=method, feat_num=fn, norm=True, ROC_sel=True, importance_sel=False, n_jobs=-1)
    
    randomforest_gini = DHL_feature_selector()
    randomforest_gini.filter_method( est=Model_ensemble_randomforest_gini, X=X, y=y, cv=cvv, meth=method, feat_num=fn, norm=True, ROC_sel=True, importance_sel=False, n_jobs=-1)

    discriminant_QDA = DHL_feature_selector()
    discriminant_QDA.filter_method( est=Model_discriminant_QDA, X=X, y=y, cv=cvv, meth=method, feat_num=fn, norm=True, ROC_sel=True, importance_sel=False, n_jobs=-1)

    with pd.ExcelWriter(savename) as writer:
        
        svc_linear.filt_score_dp.to_excel(writer, sheet_name='svc_linear_score')
        svc_linear.filt_count_dp.to_excel(writer, sheet_name='svc_linear_count')

        randomforest_gini.filt_score_dp.to_excel(writer, sheet_name='randomforest_gini_score')
        randomforest_gini.filt_count_dp.to_excel(writer, sheet_name='randomforest_gini_count')
        
        discriminant_QDA.filt_score_dp.to_excel(writer, sheet_name='discriminant_QDA_score')
        discriminant_QDA.filt_count_dp.to_excel(writer, sheet_name='discriminant_QDA_count')

Counter({0: 49, 1: 41})
Counter({0: 49, 1: 44})
Counter({1: 44, 0: 41})
